# Pricing an Interest-Rate Swap using a binomial tree: 

In this module, I price an interest-rate swap. Swaps are liquidly traded securities by which two parties exchange the cash flows. Usually one leg of the swap is fixed and the other is floating. Here I will focus on pricing a swap on the interest rate with a fixed interest rate which is paid in arrears.

We price the swap using Martingale pricing formula:

$\large S_t = \mathbb{E}[\frac{S_{t+1} + C_{t+1}}{1 + r_t}]$

where $S_t$ is the value of the swap at time t, $C_t$ is the coupon payment (cash flow) at time t and $r_t$ is the short rate. To find the price of the swap we model it using a binomial tree. 

The function swap_price takes in the following parameters: <br/>
p: the notional amount <br/>
k: the fixed rate <br/>
r_0: the short rate at time 0 <br/>
n: the expiry period of the swap <br/>
u,d: up and down factors by which the short rate evolves <br/>

To model the short rate, we will use the function short_rate, which models the evolution of the short rate using a binomial tree.

In [1]:
import numpy as np

In [82]:
def swap_price(p,k,r_0,n,u,d):
    
    #inputs:
    #p: the notional amount
    #k: the fixed rate (enter amount in %)
    #r_0: the short rate at time 0 
    #n: the expiry period of the swap 
    #u,d: up and down factors by which the short rate evolves
    
    rate_lattice = short_rate(r_0,n-1,u,d)
    
    rate_lattice /= 100.0
    
    swap_price = np.zeros_like(rate_lattice) #create an empty array of same dimension
                                             #as rate_lattice
    
    #price the terminal node of the lattice
    
    for i in range(n):
        swap_price[n-1,i] = (rate_lattice[n-1,i] - k/100.0)/(1+rate_lattice[n-1,i])
        
    #price the remaining nodes using the martingale property and risk-neutral probabilities
    
    q = 0.5    #the risk-neutral probability
      
    for i in range(n-2,-1,-1):
        for j in range(i+1):
            if i < j:
                swap_price[i,j] = ''
            else:
                swap_price[i,j] = (1/(1+rate_lattice[i,j]))* \
                                  (q*swap_price[i+1,j+1]+(1-q)*swap_price[i+1,j] + \
                                  (rate_lattice[i,j] - k/100.0))
    
    print("The swap price of a {} % swap on $ {} notional, expiring in {} periods is:".format(k,p,n))
    
    
    return swap_price[0,0]*p

In [84]:
swap_price(100,5.0,6.0,10,1.25,0.9)

The swap price of a 5.0 % swap on $ 100 notional, expiring in 10 periods is:


19.599784745960417

In [73]:
def short_rate(r_0,n,u,d):
    
    #S_0 = initial stock price
    #n: number of periods
    #u,d = up/down movement in each step
    
    rate = np.zeros((n+1,n+1))
    rate[0,0] = r_0
    
    for i in range(1,n+1):
        rate[i,0] = rate[i-1,0]*d
        for j in range(1,i+1):
            rate[i,j] = rate[i-1,j-1]*u
        
        
    return rate